# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model to determine optimal model performance
- Compare training and testing errors to determine if model is over or underfitting

## Let's get started

We included the code to pre-process below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

### Train-test split

Perform a train-test split with a test set of 20%.

In [2]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split

In [3]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [4]:
# A brief preview of train-test split
print(len(X_train), len(X_test), len(y_train), len(y_test))


1168 292 1168 292


### Fit the model

Fit a linear regression model and apply the model to make predictions on test set

In [5]:
from sklearn.linear_model import LinearRegression


linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [6]:

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

### Residuals and MSE

Calculate the residuals and the mean squared error on the test set

In [7]:
from sklearn.metrics import mean_squared_error
train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print(f'Training MSE {train_mse}')
print(f'Test MSE {test_mse}')

Training MSE 0.160749978673047
Test MSE 0.1700440367123


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function `kfolds()` that splits a dataset into k evenly sized pieces. If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [8]:
def kfolds(data, k):
    """return a list of k subsets of data"""
    # Force data as pandas DataFrame
    # add 1 to fold size to account for leftovers
    
    split_size = len(data) // k 
    split_remainder = len(data) % k
    
    folds = []
    start = 0
    end = 0
    
    for i in range(k):
        if split_remainder > 0:
            end = start + split_size + 1
            split_remainder -= 1
        else:
            end = start + split_size
        folds.append(data[start:end])
        start = end
    
    return folds

### Apply it to the Ames Housing data

In [9]:
# Make sure to concatenate the data again
ames_data = pd.concat([X, y], axis=1)

In [10]:
# Apply kfolds() to ames_data with 5 folds
folds = kfolds(ames_data, 5)
folds

[     LotArea_log  1stFlrSF_log  GrLivArea_log  BldgType_2fmCon  \
 0      -0.133185     -0.803295       0.529078                0   
 1       0.113403      0.418442      -0.381715                0   
 2       0.419917     -0.576363       0.659449                0   
 3       0.103311     -0.439137       0.541326                0   
 4       0.878108      0.112229       1.281751                0   
 ..           ...           ...            ...              ...   
 287    -0.208982     -0.795950      -1.538509                0   
 288     0.156994     -0.645537      -1.395230                0   
 289    -0.070186     -1.445511      -0.079173                0   
 290     1.053039     -0.074628       0.874786                0   
 291    -0.898448     -0.522097       0.539579                1   
 
      BldgType_Duplex  BldgType_Twnhs  BldgType_TwnhsE  KitchenQual_Fa  \
 0                  0               0                0               0   
 1                  0               0         

### Perform a linear regression for each fold and calculate the training and test error

Perform linear regression on each and calculate the training and test error: 

In [13]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
#     train = pd.concat(folds[:n-1] + folds[n+1:])
    train = pd.concat([fold for i, fold in enumerate(folds) if i!=n])
    print(len(train))
    test = folds[n]
    # Fit a linear regression model
    linreg = LinearRegression()
    linreg.fit(train.drop('SalePrice_log', axis=1), train['SalePrice_log'])
    y_hat_train = linreg.predict(train.drop('SalePrice_log', axis=1))
    y_hat_test = linreg.predict(test.drop('SalePrice_log', axis=1))
    # Evaluate Train and Test errors
    train_mse = mean_squared_error(train['SalePrice_log'], y_hat_train)
    test_mse = mean_squared_error(test['SalePrice_log'], y_hat_test)
    train_errs.append(train_mse)
    test_errs.append(test_mse)

print(train_errs)
print('')
print(test_errs)
print('')
print(f'Mean train error {sum(train_errs) / len(train_errs)}')
print(f'Mean test error {sum(test_errs) / len(test_errs)}')

1168
1168
1168
1168
1168
[0.17170509651464658, 0.15507935685930538, 0.156599463262233, 0.16134557666308721, 0.1516504855313168]

[0.12431546148437417, 0.1935006463131315, 0.18910530431311198, 0.17079325250026922, 0.20742704588916958]

Mean train error 0.15927599576611778
Mean test error 0.1770283421000113


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [14]:
from sklearn.model_selection import cross_val_score

In [25]:

linreg = LinearRegression()

cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring='neg_mean_squared_error')
cv_5_results

array([-0.12431546, -0.19350065, -0.1891053 , -0.17079325, -0.20742705])

Next, calculate the mean of the MSE over the 5 cross-validation and compare and contrast with the result from the train-test split case.

In [26]:
np.mean(cv_5_results)

-0.1770283421000113

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are robust and always use it whenever possible! 